In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim

In [ ]:

def fsin(f):
    rand = np.random.random(size=5)
    x = np.linspace(-1, 1, 32*4)
    y = np.sin((x+rand[0]) * f) * rand[1] + rand[2]
    return y
    

In [ ]:
class sorq(Dataset):
    def __init__(
            self,
            length=1000,
            device="cuda:0"
    ):
        self.length = length
        self.device = device

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        rands = np.random.rand(2)
        output = fsin(index%19) * rands[0] + rands[1]*3
        return (torch.tensor(output).view(-1,1).to(self.device).to(torch.float),
                torch.full((len(output),),index%19).view(-1,1).to(self.device).to(torch.float))

    def __str__(self) -> str:
        return f"{self.length}"

dataset = sorq(length=1_000_000)

In [ ]:
for i in range(0,20):
    # print(dataset[i][1][0].item())
    plt.plot(dataset[i][0].detach().cpu())
plt.show()

In [ ]:
class LSTMDiscriminatorRF(nn.Module):
    """An LSTM based discriminator. It expects a sequence as input and outputs a probability for each element. 
    Args:
        in_dim: Input noise dimensionality
        n_layers: number of lstm layers
        hidden_dim: dimensionality of the hidden layer of lstms
    Inputs: sequence of shape (batch_size, seq_len, in_dim)
    Output: sequence of shape (batch_size, seq_len, 1)
    """

    def __init__(self, in_dim,out_dim = 19, n_layers=1, hidden_dim=256):
        super().__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim
        self.lstm = nn.LSTM(in_dim, hidden_dim, n_layers, batch_first=True)
        self.linear_recurrent = nn.Sequential(nn.Linear(hidden_dim, out_dim), nn.Sigmoid())

    def forward(self, input):
        batch_size, seq_len = input.size(0), input.size(1)
        h_0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to("cuda:0")
        c_0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to("cuda:0")

        recurrent_features, (h_out,c_out) = self.lstm(input, (h_0, c_0))
        outputs = self.linear_recurrent(recurrent_features.contiguous().view(batch_size*seq_len, self.hidden_dim))
        outputs = outputs.view(batch_size, seq_len, self.out_dim)
        return outputs

In [ ]:
batch_size = 16
input_dimenstions = 1
layers = 4
hidden_size = 1024
dataloader = DataLoader(dataset,batch_size=16,shuffle = True)

disc1 = LSTMDiscriminatorRF(input_dimenstions,n_layers=layers,hidden_dim=hidden_size).to("cuda:0")
optimizer1 = optim.Adam(disc1.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss().to(dataset.device)


In [ ]:
print("epoch |  bnum  | errD1  |")
for epoch in range(30):
    for i, (data,label) in enumerate(dataloader, 0):
        # print(data.device,label.device)
        disc1.zero_grad()
        output = disc1(data)
        coutput = output[:,-1:].view(16,19)
        clabel = label[:,-1:].view(16).to(torch.long)
        # print(coutput.shape,clabel.shape)
        errD1 = criterion(coutput,clabel)
        errD1.backward()
        optimizer1.step()

        print(f"{epoch:5} | {i:6} | {errD1.item():6.3}",end = "\r",flush=True)
        if i%100 == 0:
            print()
    print("\nepoch finished: ",epoch)

In [ ]:
c,label = next(iter(DataLoader(dataset=dataset,batch_size=len(dataset),shuffle=True)))
output = disc1(c)
plabel = label[:,0,:]
output = output.view(-1,1)
print(plabel.shape, output.shape)
corrects = []
for i,j in zip(plabel,output):
    print(i.item(),round(j.item()))
    corrects.append(int(i.item() == j.item()))
